<img src="./IMTA.png" alt="Logo IMT Atlantique" width="300"/>

##  **Introduction à PyTorch/MONAI - Structuration d’un projet de Deep Learning**
## TAF Health - UE B - 2025/2026 

Pierre-Henri.Conze@imt-atlantique.fr - Vincent.Jaouen@imt-atlantique.fr


**Objectifs** 

- Manipuler des images médicales avec MONAI et PyTorch.
-  Comprendre les grandes étapes d’un code d’apprentissage profond 
   - Préparation des données (dataloader, transforms)
   - Définition du modèle
   - Boucle d’entraînement
   - Validation et inférence
- Mettre en place de bonnes pratiques de développement en machine learning : 
   - Utilisation d'environnement virtuel
   - Structuration du code avec des modules réutilisables (code factoring)

**Introduction à MONAI**

[MONAI](https://monai.io/) (Medical Open Network for AI) est une librairie open-source conçue pour le **deep learning en imagerie médicale**.  
Développée à l’origine par des équipes de NVIDIA et King’s College London, elle est devenue rapidement la **référence** pour la recherche et le développement en IA appliquée à la santé.

MONAI est bâtie **au-dessus de [PyTorch](https://pytorch.org/)** :  
- elle hérite donc de la flexibilité et de la puissance de PyTorch (tensors, GPU, backpropagation, etc.),  
- mais ajoute une surcouche spécialisée pour les **besoins spécifiques de l’imagerie médicale**.

---

### Pourquoi MONAI en IA médicale ?

- **Spécialisée pour l’imagerie 3D** : contrairement aux jeux de données classiques (ImageNet, photos 2D), l’imagerie médicale est souvent volumique (IRM, scanner, TEP). MONAI gère nativement ces formats.  
- **Transforms dédiés** : des opérations adaptées (coupes 3D, interpolations médicales, normalisations d’intensité, etc.) qui ne sont pas disponibles dans PyTorch "pur".  
- **Formats standards** : compatibilité directe avec les formats de données cliniques (DICOM, NIfTI, etc.).  
- **Exemples cliniques** : de nombreux modèles utilisés en imagerie médicale sont disponibles, pour la segmentation de tumeurs, détection de lésions, reconstruction, classification.  
- **Communauté et adoption** : soutenue par NVIDIA, des hôpitaux, des universités et des industriels, MONAI est devenu un standard de facto pour les publications et les projets académiques.

---

### MONAI et PyTorch : un duo complémentaire

- **PyTorch** est le moteur : calcul différentiable, optimisation, gestion GPU/CPU.  
- **MONAI** est la boîte à outils spécialisée : chargement des images médicales, prétraitements adaptés, réseaux neurones 3D prêts à l’emploi, métriques médicales.  

En résumé : **MONAI = PyTorch + expertise médicale intégrée**.  

## Installation de l’environnement MONAI (CPU-only) local

Pour ce TP, nous créons un environnement Python **local à la machine TP** dans `/users/local/monai`.  
Ce répertoire est **en écriture et rapide** (disque local), contrairement au `$HOME` sur NFS qui est lent.

1. **Créer l’environnement virtuel**
   ```bash
   $ python3 -m venv /users/local/monai

2. **Activer l'environnement**
    ```bash
    $ source /users/local/monai/bin/activate
Votre prompt devrait afficher `(monai)`.

3. **Installer les dépendances : pytorch version cpu, monai, tools jupyter**
    ```bash
    $ pip install --index-url https://download.pytorch.org/whl/cpu torch torchvision torchaudio
    $ pip install monai[all]
    $ pip install ipykernel jupyterlab tqdm matplotlib 

4. **Enregistrer le kernel du venv dans Jupyter (utilisateur courant)**
    ```bash
    $ python -m ipykernel install --user --name=monai --display-name "monai"

4. **Lancer JupyterLab et sélectionner le bon kernel**
    ```bash
    $ jupyter-lab
Dans JupyterLab, choisissez le kernel Python (monai) pour vos notebooks.

**Réactivation rapide (session suivante)**

À chaque nouvelle session (pas besoin de réinstaller) 
```bash
   $ source /users/local/monai/bin/activate
   $ jupyter-lab
   ```
**Désactivation (optionel)**

Pour retourner au shell
```bash
    $ deactivate
```



# Partie 1 – Prise en main de MONAI
Commençons par charger une image IRM 3D avec MONAI et faisons des manipulations simples.

In [ ]:

import torch
from monai.transforms import LoadImage
import matplotlib.pyplot as plt

# TODO : change path to a NIfTI file
img_path_t1ce = "../datasets/3d_examples/BraTS-GLI-00000-000-t1c.nii.gz"
img_path_t2 = "../datasets/3d_examples/BraTS-GLI-00000-000-t2w.nii.gz"

loader = LoadImage(image_only=True)
img = loader(img_path_t1ce)

print("Shape:", img.shape)

# Display one axial slice
plt.imshow(img[:,:,img.shape[2]//2], cmap="gray")
plt.title("Coupe axiale")
plt.show()


**Exercice 1.1** 

Cette image a été produite par IRM pondérée en T1 avec injection de produit contrastant au gadolinium. Le gadolinium est un agent paramagnétique particulièrement intéressant pour l'augmentation du contraste, même si il présente des inconvénients (toxicité, polluant éternel). 

Elle montre un glioblastome cérébral dans la zone frontale du cerveau du patient (à gauche sur cette coupe axiale).

👉 **Questions** : 
1. Afficher une coupe sagittale et coronale au niveau de cette tumeur
2. Afficher l'image de pondération T2. Pourquoi les contrastes sont inversés ?
2. Changer la colormap de l'affichage
3. Donner la valeur du voxel aux coordonnées (120,120,76)


## Transforms

### Exercice 1.2 — Exploration des intensités et premières transforms

Dans MONAI, on utilise des **`transforms`** pour appliquer des pré-traitements sur les images médicales :  
- `LoadImaged` : charge une image depuis le disque,  
- `ScaleIntensityd` : ajuste l’échelle des intensités,  
- `NormalizeIntensityd` : centre et normalise les intensités,  
- `ToTensord` : convertit les données en tenseurs PyTorch.  

👉 La documentation complète des transforms est disponible ici : [MONAI Transforms](https://docs.monai.io/en/stable/transforms.html).  

---




In [ ]:
from monai.transforms import LoadImaged, ScaleIntensityd, Compose

# Exemple minimal : on définit un pipeline de transforms
transforms = Compose([
    LoadImaged(keys=["t1ce"]),         # Chargement
    ScaleIntensityd(keys=["t1ce"])     # Mise à l’échelle des intensités
])

# Application de la séquence de transforms
data_dict = {"t1ce": img_path_t1ce}
processed = transforms(data_dict)

print("Type après transform :", type(processed["t1ce"]))
print("Forme de l'image :", processed["t1ce"].shape)

👉 **Questions** : 

1. Inspirez-vous de l’exemple ci-dessus pour appliquer un pipeline de transforms qui :
   - charge l’image,
   - normalise les intensités (`NormalizeIntensityd`),

2. Affichez l’histogramme des intensités **avant et après normalisation**.  
   *Aide : utilisez `plt.hist(processed["image"].flatten(), bins=100)`.*

3. Quelle est la nouvelle valeur du voxel aux coordonnées `(120, 120, 76)` après normalisation ?

4. Comparez les intensités moyennes entre :
   - tissu cérébral sain,
   - région tumorale.  
   *(Indice : sélectionnez des régions rectangulaires simples pour l’instant.)*


## Dataset et DataLoader

En deep learning, on manipule généralement **un grand nombre de fichiers** (images, labels, masques, etc.).  
Il est donc nécessaire d’avoir une organisation systématique pour :
- **accéder aux données** (charger depuis le disque),
- **appliquer des pré-traitements** (normalisation, resize, etc.),
- **les regrouper par lots (batchs)** pour les envoyer au réseau de neurones.

PyTorch fournit deux briques essentielles :

- **`Dataset`** : représente une *collection de données* (par exemple la liste des couples *image + label*).  
- **`DataLoader`** : enveloppe le dataset et permet d’itérer dessus efficacement par *batchs* (gestion du parallélisme, mélange aléatoire, etc.).

---

### Cas de l’imagerie médicale

En imagerie médicale, les entrées sont souvent **couplées** :  
- une **image médicale** (IRM, scanner, etc.),  
- un **label ou masque associé** (segmentation, classification).  

Par exemple, pour un problème de segmentation tumorale, chaque **image IRM** est associée à un **masque de la tumeur**.  

C’est ce que nous allons mettre en place ici avec MONAI.




## Dataset et DataLoader (version simple)

Avant d’aller plus loin, créons un premier **Dataset** qui relie directement :
- le chemin du fichier image (`img_fname`),
- le chemin du fichier de segmentation (`seg_fname`).

Cela correspond à la manière la plus simple d’organiser nos données pour un problème de **segmentation d’images médicales**.

---

### Exemple de JSON de description des données

Un fichier `dataset.json` contient une liste d’entrées au format :

```json
{
  "training": [
    {
      "img": "/chemin/vers/img1.nii.gz",
      "seg": "/chemin/vers/seg1.nii.gz"
    },
    {
      "img": "/chemin/vers/img2.nii.gz",
      "seg": "/chemin/vers/seg2.nii.gz"
    }
  ]
}

Un exemple de dataset.json déjà construit est disponible dans le dossier ../datasets/MidTumors/ reliant :
- une image `trainA/BraTS_****_3c.nii.gz` 
- à un masque de segmentation `trainB/BraTS_****_seg.nii.gz`

In [ ]:

from monai.transforms import (
    LoadImaged, EnsureChannelFirstd, Resized,
    ScaleIntensityRangePercentilesd, EnsureTyped, Compose
)
from monai.data import Dataset, DataLoader
import os, json

# Répertoire des données (contenant dataset.json et les sous-dossiers trainA/trainB)
data_dir = "../datasets/MidTumors"
with open(os.path.join(data_dir, "dataset.json")) as f:
    data = json.load(f)
    
# Définition des transforms de base
from monai.transforms import Compose, LoadImaged, EnsureChannelFirstd, ScaleIntensityRangePercentilesd, EnsureTyped, Lambdad

train_transforms = Compose([
    LoadImaged(keys=["image", "label"]),
    EnsureChannelFirstd(keys=["image", "label"]),   # (H,W,3) → (1,H,W,3)
    ScaleIntensityRangePercentilesd(
        keys="image", lower=1, upper=99, b_min=-1.0, b_max=1.0, clip=True
    ),
    # Reorder dimensions: (C=1, H, W, 3) → (3, H, W)
    Lambdad(
        keys=["image","label"],
        func=lambda x: x.permute(3,1,2,0).squeeze(-1)   # (1,H,W,3) → (3,H,W)
    ),
    EnsureTyped(keys=["image", "label"]),
])

# On crée les chemins complets depuis dataset.json
train_files = [
    {
        "image": os.path.join(data_dir, f["image"]),
        "label": os.path.join(data_dir, f["label"])
    }
    for f in data["training"]   # juste 10 exemples pour l’énoncé
]

# Création Dataset + DataLoader
train_ds = Dataset(data=train_files, transform=train_transforms)
train_loader = DataLoader(train_ds, batch_size=1, shuffle=True)

# Test d’une itération
batch = next(iter(train_loader))
print(batch["image"].shape, batch["label"].shape)

👉 **Question** : Analysez l'image chargée par le modèle : 

- Déterminez sa dimension. 
- Visualisez les 3 canaux 

En fait d'image tridimensionnelle, ``image`` est la même coupe axiale d'un patient visualisée par trois séquences IRM différentes. Il est courant en IRM d'observer plusieurs types de contrastes. 

Un MONAI tensor doit : 
- en 3D, avoir des dimensions $N_{batchsize}\times N_{channels} \times N_x \times N_y\times N_z$
- 2D doit, avoir des dimensions $N_{batchsize}\times N_{channels} \times N_x \times N_y$


## Partie 3 – Modèle et entraînement

Entraînons désormais un modèle de segmentation sur ces trois coupes. 

Nous devons définir un DataLoader d'un Dataset que nous définirons (cf TP précédent)

In [ ]:
from monai.networks.nets import UNet
from monai.losses import DiceCELoss
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- Define model ---
model = UNet(
    spatial_dims=2,
    in_channels=3,     # RGB-like input
    out_channels=1,    # binary segmentation
    channels=(16, 32, 64),
    strides=(2, 2),
).to(device)

# --- Loss & optimizer ---
loss_fn = DiceCELoss(sigmoid=True, to_onehot_y=False)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# --- Training loop ---
max_epochs = 150
for epoch in range(max_epochs):
    model.train()
    epoch_loss = 0
    for batch in train_loader:   # train_loader must yield dicts with "image" & "label"
        inputs = batch["image"].to(device)   # shape (B,3,H,W)
        labels = batch["label"].to(device)   # shape (B,1,H,W)

        # Forward pass
        outputs = model(inputs)              # (B,1,H,W)

        # Loss
        loss = loss_fn(outputs, labels)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f"Epoch {epoch+1}/{max_epochs}, Loss = {epoch_loss/len(train_loader):.4f}")


Voilà, vous avez un modèle de segmentation qui s'entraine sur 3 canaux !

## Partie 4 – Structuration du code

Jusqu’à présent, nous avons identifié trois composants essentiels d’un projet de Deep Learning :  
- Les **transforms**, qui assurent le prétraitement et la mise en forme des données.  
- Le **DataLoader**, qui fournit des minibatches pendant l’entraînement.  
- La **boucle d’entraînement** (*training loop*), qui met à jour les poids du réseau par rétropropagation à chaque itération.  

À cela, il faut ajouter deux éléments indispensables dans toute expérimentation sérieuse :  
- **L’évaluation**, permettant de calculer des métriques de validation et de visualiser les résultats intermédiaires.  
- **La sauvegarde des poids**, afin de conserver le meilleur modèle selon un critère de performance choisi.  

---

### Vers une structuration modulaire du code

Pour rendre un code de Deep Learning fonctionnel et flexible, il est nécessaire d’adopter une organisation modulaire.  
Une structuration typique consiste à factoriser les différentes parties dans des fichiers distincts :  

- `utils/data_utils.py` : définition des loaders et des transformations.  
- `utils/training.py` : implémentation des boucles d’entraînement.  
- `utils/evaluation.py` : calcul des métriques et visualisation des résultats.  
- `inference/*.py` : scripts permettant d’appliquer un modèle entraîné à de nouvelles données.  

---

### Mise en pratique dans les notebooks

Dans ce cours, nous allons travailler à partir de trois notebooks factorisés :  

- `01_segmentation.ipynb`  
- `02_classification.ipynb`  
- `03_synthesis.ipynb`  

Ces notebooks exploitent tous le dataset **MidTumors**, que nous avons brièvement exploré.  
L’intérêt de cette factorisation est de permettre, dans un cadre commun, l’entraînement de modèles répondant à des objectifs variés :  

- **Segmentation** : prédiction d’un masque à partir des trois contrastes IRM.  
- **Classification** : identification de la modalité (T1, T2, ou FLAIR).  
- **Synthèse** : génération d’une pseudo-modalité T2 à partir d’une image T1.  

Naturellement, les architectures de réseaux de neurones employées diffèrent selon la tâche, mais la structure de code sous-jacente reste la même.  

👉 **Question** : Executez ces différents notebooks et analysez les. Identifiez les spécificités liées à chaque tâche : configuration / modèles / data loader...

---

### Ouverture

Une telle organisation ouvre la voie à de nombreuses autres tâches en adaptant les data loaders et les transforms. 

👉 **Question** : adaptez ce travail à un autre jeu de labels contenu dans `../datasets/MidTumors_3labels`, où les tumeurs sont cette fois décrites selon trois zones distinctes :  
- **œdème**,  
- **cœur tumoral (nécrose/partie solide)**,  
- **tumeur rehaussée** (*zones présentant une prise de contraste après injection de Gadolinium paramagnétique*).  

👉 **Bonus** : adaptez ce travail à une nouvelle tâche (débruitage? défloutage?)

#### Exemples possibles
- Débruitage
- Defloutage 
- Apprentissage de modèles de normalisation inter-patients (réduction des biais liés aux contrastes).  
